### Interfaz grafica para el modelo con iypywidgets (IMPORTANTE: Se le pidió a CHATGPT-5 Que le creara la interfaz grafica con el modelo, siempre estando al pendiente de lo que hacía , pidiendo excatamente lo que se necesitaba y como se necestiaba )

In [1]:
import ipywidgets as widgets
import joblib
import os
from IPython.display import display, HTML

# Carga robusta del modelo
pipline = None
model_status = widgets.HTML()

try:
    # Rutas candidatas donde podría estar el modelo
    candidate_paths = [
        'spam_detect.pkl',
        './spam_detect.pkl',
        '../spam_detect.pkl',
    ]
    found_path = next((p for p in candidate_paths if os.path.exists(p)), None)
    if found_path is None:
        raise FileNotFoundError('spam_detect.pkl no encontrado en rutas conocidas')

    pipline = joblib.load(found_path)
    model_status.value = '<b style="color:#2e7d32;">Modelo cargado exitosamente ✅</b>'
except FileNotFoundError as e:
    model_status.value = f'<b style="color:#c62828;">No se encontró el modelo: {e}</b>'
except Exception as e:
    model_status.value = f'<b style="color:#c62828;">Error al cargar el modelo: {e}</b>'

display(model_status)

HTML(value='<b style="color:#2e7d32;">Modelo cargado exitosamente ✅</b>')

In [2]:
import re
from IPython.display import display

# Paleta y estilos
primary = '#1976d2'
success = '#2e7d32'
warning = '#f9a825'
error = '#c62828'
neutral_bg = '#f5f5f5'
card_bg = '#ffffff'

app_title = widgets.HTML(
    value=f"""
    <div style='background:{primary};color:white;padding:12px 16px;border-radius:8px;font-size:18px;'>
        📧 Detección de Spam — Demo interactiva
    </div>
    """
)

# Aviso en verde sobre el posible error del modelo y el idioma
info_banner = widgets.HTML(
    value=f"""
    <div style='background:#e8f5e9;color:{success};padding:10px 12px;border-left:4px solid {success}; border-radius:8px;margin-top:6px;'>
        <b>Nota:</b> Este modelo es probabilístico y puede equivocarse. Ajusta el <i>umbral</i> según tu tolerancia al error.<br/>
        Si la probabilidad está cerca del umbral (±5 pp), revisa la predicción manualmente.<br/>
        <b>Importante:</b> el modelo fue entrenado con mensajes en <u>inglés</u>; para mejores resultados, ingresa el texto en inglés o tradúcelo antes de clasificar.
    </div>
    """
)

# Widgets de entrada
text_input = widgets.Textarea(
    placeholder='Escribe aquí el mensaje a analizar...',
    description='Mensaje',
    layout=widgets.Layout(width='100%', height='120px')
)

toggle_proba = widgets.Checkbox(
    value=True,
    description='Usar probabilidad (si el modelo la soporta)',
)

threshold = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=0.9,
    step=0.05,
    description='Umbral',
    readout_format='.2f',
    layout=widgets.Layout(width='50%')
)

classify_btn = widgets.Button(
    description='Clasificar',
    button_style='primary',
    icon='search',
    layout=widgets.Layout(width='150px')
)

clear_btn = widgets.Button(
    description='Limpiar',
    icon='trash',
    layout=widgets.Layout(width='120px')
)

# Salidas
prediction_box = widgets.HTML(
    value=f"<div style='padding:12px;border-radius:8px;background:{neutral_bg};color:#333;'>Esperando entrada…</div>"
)

details_box = widgets.HTML(
    value=""
)

# Helpers

def _supports_proba(model):
    return hasattr(model, 'predict_proba') or (
        hasattr(model, 'named_steps') and any(
            hasattr(step, 'predict_proba') for step in model.named_steps.values()
        )
    )


def _predict(text):
    if pipline is None:
        return None, None, 'El modelo no está cargado.'

    # Normalización mínima del texto (opcional, el pipeline debería manejarlo)
    clean = re.sub(r'\s+', ' ', text).strip()
    if not clean:
        return None, None, 'Ingresa un mensaje válido.'

    proba = None
    if toggle_proba.value and _supports_proba(pipline):
        try:
            # predict_proba devuelve [ [p_no_spam, p_spam] ] típicamente
            proba = pipline.predict_proba([clean])[0]
            # Asumimos clase positiva = índice 1
            p_spam = float(proba[1]) if len(proba) > 1 else float(proba[0])
            label = int(p_spam >= threshold.value)
            return label, p_spam, None
        except Exception as e:
            # Caer a predicción sin probas
            pass

    try:
        label = int(pipline.predict([clean])[0])
        return label, proba, None
    except Exception as e:
        return None, None, f'Error en predicción: {e}'


def on_classify(_):
    label, p_spam, err = _predict(text_input.value)
    if err:
        prediction_box.value = f"<div style='padding:12px;border-radius:8px;background:{neutral_bg};color:{error};'>🚫 {err}</div>"
        details_box.value = ""
        return

    if label is None:
        prediction_box.value = f"<div style='padding:12px;border-radius:8px;background:{neutral_bg};color:{error};'>🚫 No se pudo generar una predicción.</div>"
        details_box.value = ""
        return

    if label == 1:
        bg = '#ffebee'
        col = error
        tag = 'SPAM'
        icon = '⚠️'
    else:
        bg = '#e8f5e9'
        col = success
        tag = 'NO SPAM'
        icon = '✅'

    proba_html = ''
    if p_spam is not None:
        pct = int(round(p_spam * 100))
        margin_pp = int(round(abs(p_spam - threshold.value) * 100))
        near_msg = ' — Revisa manualmente' if margin_pp <= 5 else ''
        bar_inner = f"<div style='width:{pct}%;height:100%;background:{col};border-radius:6px;'></div>"
        proba_html = f"""
        <div style='margin-top:8px;'>
            <div style='font-size:12px;color:#555;'>Probabilidad de SPAM: <b>{pct}%</b></div>
            <div style='width:100%;height:10px;background:#eee;border-radius:6px;overflow:hidden;'>{bar_inner}</div>
            <div style='margin-top:6px;color:{success};font-size:12px;'>Margen al umbral: <b>{margin_pp} pp</b>{near_msg}</div>
        </div>
        """

    prediction_box.value = f"""
    <div style='background:{bg};color:{col};padding:16px;border-radius:10px;font-size:16px;'>
        <div style='display:flex;align-items:center;gap:8px;'>
            <span>{icon}</span>
            <b>Resultado:</b> {tag}
        </div>
        {proba_html}
    </div>
    """

    # Detalles del mensaje (preview)
    preview = text_input.value.strip()
    preview = preview if len(preview) <= 240 else preview[:240] + '…'
    details_box.value = f"""
    <div style='background:{card_bg};padding:12px;border-radius:10px;border:1px solid #eee;margin-top:8px;'>
        <div style='font-size:12px;color:#444;margin-bottom:6px;'>Vista previa del mensaje</div>
        <div style='white-space:pre-wrap;color:#222;'>{preview}</div>
    </div>
    """


def on_clear(_):
    text_input.value = ''
    prediction_box.value = f"<div style='padding:12px;border-radius:8px;background:{neutral_bg};color:#333;'>Esperando entrada…</div>"
    details_box.value = ''


classify_btn.on_click(on_classify)
clear_btn.on_click(on_clear)

# Layout
controls_row = widgets.HBox([
    classify_btn, clear_btn, widgets.Layout(width='8px')
])

options_row = widgets.HBox([
    toggle_proba, threshold
])

input_card = widgets.VBox([
    widgets.HTML("<b>Ingresa el mensaje</b>"),
    text_input,
    options_row,
    controls_row
], layout=widgets.Layout(padding='12px', background=card_bg, border='1px solid #eee', border_radius='10px'))

output_card = widgets.VBox([
    widgets.HTML("<b>Predicción</b>"),
    prediction_box,
    details_box
], layout=widgets.Layout(padding='12px', background=card_bg, border='1px solid #eee', border_radius='10px'))

app = widgets.VBox([
    app_title,
    info_banner,
    input_card,
    output_card
], layout=widgets.Layout(gap='10px'))

display(app)